In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1610044156730_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  r.review_headline, r.review_comments, \
case when (contains(r.observations, 'PR') )  then 1 else 0 end as PR \
from analytics.review as r \
where r.locale in ('en_US') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR')\
and r.created_date > '2020-01-01' "


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head(20)
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID  ... PR
0   318768462  ...  0
1   318774873  ...  0
2   318778726  ...  0
3   318782481  ...  0
4   318779118  ...  0
5   318784187  ...  1
6   318785045  ...  0
7   318754976  ...  0
8   318772438  ...  0
9   318782529  ...  0
10  318712617  ...  0
11  318701542  ...  0
12  318697986  ...  0
13  318704811  ...  0
14  318774294  ...  0
15  318781066  ...  0
16  318726498  ...  1
17  318692799  ...  0
18  318783687  ...  0
19  318774793  ...  0

[20 rows x 4 columns]

In [6]:

reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ...                                           ALL_TEXT
0  318768462  ...  Nautical sun suit I love the navy and white st...
1  318774873  ...  It was great, I was looking for the heritage f...
2  318778726  ...                          Good material Confortable
3  318782481  ...  Relaxed fit very comfortable I love the fit of...
4  318779118  ...  sooo soft! Great texture, very soft and good q...

[5 rows x 5 columns]

In [7]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6504876 entries, 0 to 6504875
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UGC_ID           int64 
 1   REVIEW_HEADLINE  object
 2   REVIEW_COMMENTS  object
 3   PR               int64 
 4   ALL_TEXT         object
dtypes: int64(2), object(3)
memory usage: 248.1+ MB

In [8]:
#First remove price range references that use $
#reviews['regex_range_$'] = reviews['ALL_TEXT'].str.replace(r'((OVER|UNDER|AROUND|ALMOST|ABOUT|APPROXIMATELY|APPROX|UPWARDS OF|NORTH OF|SOUTH OF|LESS THAN|MORE THAN)\s\$[0-9,.]+)|(\$[0-9,.]+\s?(\-|(TO))\s?\$?[0-9,.]+)|(\$[0-9,.]+\s(RANGE|OR MORE|OR LESS|OR SO))|(BETWEEN\s\$[0-9,.]+\sAND\s\$?[0-9,.]+)', '', case=False)
reviews['regex_range_$'] = reviews['ALL_TEXT'].str.replace(r'((OVER|UNDER|AROUND|ALMOST|ABOUT|APPROXIMATELY|APPROX|CLOSE TO|NEARLY|UPWARDS OF|NORTH OF|SOUTH OF|LESS THAN|MORE THAN|>|<)\s?\$[0-9,.]+)|(\$[0-9,.]+\s?(\-|(TO))\s?\$?[0-9,.]+)|(\$[0-9,.]+\s?(RANGE|OR MORE|OR LESS|OR SO|MORE|LESS|OFF|COUPON|DISCOUNT|\+))|(BETWEEN\s\$[0-9,.]+\sAND\s\$?[0-9,.]+)', '', case=False)


#Next remove price range references that use word "dollar" or "buck"
#reviews['regex_range_dollarbuck'] = reviews['regex_range_$'].str.replace(r'((OVER|UNDER|AROUND|ALMOST|ABOUT|APPROXIMATELY|APPROX|UPWARDS OF|NORTH OF|SOUTH OF|LESS THAN|MORE THAN)\s[0-9,.]+\s(DOLLAR|BUCK)S?)|([0-9,.]+\s?(\-|(TO))\s?[0-9,.]+\s(DOLLAR|BUCK)S?)|([0-9,.]+\s(DOLLAR|BUCK)S?\s(RANGE|OR MORE|OR LESS|OR SO))|(BETWEEN\s[0-9,.]+\s?(DOLLAR|BUCK)?S?\sAND\s[0-9,.]+(\sDOLLAR|\sBUCK)S?)', '', case=False)
reviews['regex_range_dollarbuck'] = reviews['regex_range_$'].str.replace(r'((OVER|UNDER|AROUND|ALMOST|ABOUT|APPROXIMATELY|APPROX|CLOSE TO|NEARLY|UPWARDS OF|NORTH OF|SOUTH OF|LESS THAN|MORE THAN|<|>)\s?[0-9,.]+\s?(DOLLAR|BUCK)S?)|([0-9,.]+\s?(\-|(TO))\s?[0-9,.]+\s(DOLLAR|BUCK)S?)|([0-9,.]+\s(DOLLAR|BUCK)S?\s(RANGE|OR MORE|OR LESS|OR SO|MORE|LESS|OFF|COUPON|DISCOUNT))|(BETWEEN\s[0-9,.]+\s?(DOLLAR|BUCK)?S?\sAND\s[0-9,.]+\s(DOLLAR|BUCK)S?)|([0-9,.]+\+\s(DOLLAR|BUCK)S?)', '', case=False)

#Next remove price mentions relating to shipping
reviews['regex_shipping_$'] = reviews['regex_range_dollarbuck'].str.replace(r'\$[0-9,.]+(\s?([A-Z]+)?){0,4}\s(POSTAGE|SHIPPING|SHIP|DELIVER)', '', case=False)

reviews['regex_shipping_dollarbuck'] = reviews['regex_shipping_$'].str.replace(r'[0-9,.]+\s?(DOLLAR|BUCK)S?(\s?([A-Z]+)?){0,4}\s(POSTAGE|SHIPPING|SHIP|DELIVER)', '', case=False)

#Then look for any that still have a price reference and mark as PR
#Need to require digit first, then any combo of digit, comma, or period. Otherwise would match "Not worth the $$."
reviews['regex_price'] = reviews['regex_shipping_dollarbuck'].str.contains(r'(\$\d([0-9,.]+)?)|(\d([0-9,.]+)?\s?(DOLLAR|BUCK|\$)S?)', case=False)==True

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:3: FutureWarning: The default value of regex will change from True to False in a future version.
<string>:8: FutureWarning: The default value of regex will change from True to False in a future version.
<string>:11: FutureWarning: The default value of regex will change from True to False in a future version.
<string>:13: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [9]:
reviews['regex_PR'] = 0
reviews.loc[(reviews['regex_price']==True) , 'regex_PR'] = 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... regex_PR
0  318768462  ...        0
1  318774873  ...        0
2  318778726  ...        0
3  318782481  ...        0
4  318779118  ...        0

[5 rows x 11 columns]

In [11]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#reviews.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [12]:
FP = reviews[(reviews['PR'] == 0) & (reviews['regex_PR'] == 1)]
FN = reviews[(reviews['PR'] == 1) & (reviews['regex_PR'] == 0)]
TP = reviews[(reviews['PR'] == 1) & (reviews['regex_PR'] == 1)]
TN = reviews[(reviews['PR'] == 0) & (reviews['regex_PR'] == 0)]
FP['UGC_ID'].count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22729

In [13]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8374

In [14]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

33370

In [15]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6440403

In [16]:
FN.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         UGC_ID                                               REVIEW_HEADLINE  \
1835  320877083  I would never recommend timberland brand for my family and f   
1952  320341339  There's a better choice                                        
2074  320341516  I am disappointed at the moment.                               
3742  312217287  Dont waste your money                                          
4875  312194992  Wouldn't buy in the first place. Price gouging                 

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [26]:
reviews['regex_decimal'] = reviews['ALL_TEXT'].str.contains(r'[0-9]+\.[0-9][0-9]\b', case=False)==True

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
test = reviews[ (reviews['regex_decimal'] == True)]
test.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID  \
951    280437621   
1804   320904027   
3268   268888742   
3590   268937981   
3707   312216040   
4155   312209754   
4177   312201299   
4481   320956465   
4669   320960878   
4999   312166531   
5223   318132327   
5445   320957210   
5471   320958033   
5838   318136478   
6250   318131600   
6418   276298288   
6537   276343946   
6559   276328599   
7474   318152045   
7508   318144618   
7511   318142140   
7587   318131607   
8114   276340939   
8614   318569596   
9173   278542559   
9252   278570314   
9299   278645649   
9338   318594787   
9444   318597296   
9532   318581305   
9744   318584660   
9878   318606125   
10211  320334589   
10219  320337600   
10979  278643685   
10999  278644082   
11011  278580974   
11522  268936236   
11614  268917169   
11644  318611829   
12072  319049431   
13731  268940646   
14032  267305530   
14067  267286797   
14286  319259015   
14468  319388000   
14743  308750726   
15021  308688412   
15520  299143564   


In [17]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("PR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("regex_range_$", StringType(), True)\
,StructField("regex_range_dollarbuck", StringType(), True)\
,StructField("regex_shipping_$", StringType(), True)\
,StructField("regex_shipping_dollarbuck", StringType(), True)\
,StructField("regex_price", BooleanType(), True)\
,StructField("regex_PR", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : 'APP_SPARK_PROD',
  'sfPassword' : '>6txhcQ*7&^Qrq`j',
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","PR_FP_test1").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FN_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FN_test2")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FP_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FP_test2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…